In [2]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [4]:
path = r"C:\Users\anish\02-2025 Instacart Basket Analysis"

In [6]:
path

'C:\\Users\\anish\\02-2025 Instacart Basket Analysis'

In [8]:
df_ords_prods_merge = pd.read_pickle(os.path.join(path, 'Data', 'Prepared Data', 'ords_prods_merge.pkl'))

## Create a price_label column (#1)

In [11]:
# Define a function for "price label"
def price_label(row): 
    if row['prices'] <= 5: return 'Low-range product'
    elif (row['prices'] > 5) and (row['prices'] <= 15): return 'Mid-range product'
    elif row['prices'] > 15:  return 'High-range product'
    else: return 'Not enough data'

In [13]:
df_ords_prods_merge.apply(price_label, axis=1)

0           Mid-range product
1           Mid-range product
2           Low-range product
3           Low-range product
4           Low-range product
                  ...        
32406036    Mid-range product
32406037    Mid-range product
32406038    Low-range product
32406039    Low-range product
32406040    Low-range product
Length: 32406041, dtype: object

In [14]:
df = df_ords_prods_merge[:1000000]

In [17]:
df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'

C:\Users\anish\AppData\Local\Temp\ipykernel_33216\1169838859.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'


In [19]:
df.loc[(df['prices'] <= 15) & (df['prices'] > 5), 'price_range_loc'] = 'Mid-range product'

In [21]:
df.loc[df['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [23]:
df['price_range_loc'].value_counts(dropna = False)

price_range_loc
Mid-range product     673194
Low-range product     314381
High-range product     12425
Name: count, dtype: int64

In [25]:
df_ords_prods_merge.loc[df_ords_prods_merge['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [27]:
df_ords_prods_merge.loc[(df_ords_prods_merge['prices'] <= 15) & (df_ords_prods_merge['prices'] > 5), 'price_range_loc'] = 'Mid-range product'

In [29]:
df_ords_prods_merge.loc[df_ords_prods_merge['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 16 columns):
 #   Column                  Non-Null Count    Dtype   
---  ------                  --------------    -----   
 0   order_id                1000000 non-null  int64   
 1   user_id                 1000000 non-null  int64   
 2   order_number            1000000 non-null  int64   
 3   orders_day_of_week      1000000 non-null  int64   
 4   orders_hour             1000000 non-null  int64   
 5   days_since_prior_order  936394 non-null   float64 
 6   product_id              1000000 non-null  int64   
 7   add_to_cart_order       1000000 non-null  int64   
 8   reordered               1000000 non-null  int64   
 9   _merge                  1000000 non-null  category
 10  product_name            1000000 non-null  object  
 11  aisle_id                1000000 non-null  int64   
 12  department_id           1000000 non-null  int64   
 13  prices                  1000000 non-null  f

In [33]:
df_ords_prods_merge['price_range_loc'].value_counts(dropna = False)

price_range_loc
Mid-range product     21861997
Low-range product     10126366
High-range product      417678
Name: count, dtype: int64

## Create busiest_day column

In [38]:
df_ords_prods_merge['orders_day_of_week'].value_counts(dropna = False)

orders_day_of_week
0    6204404
1    5660456
6    4496635
2    4213986
5    4205906
3    3840701
4    3783953
Name: count, dtype: int64

In [42]:
result = []

for value in df_ords_prods_merge["orders_day_of_week"]:
  if value == 0:
    result.append("Busiest day")
  elif value == 4:
    result.append("Least busy")
  else:
    result.append("Regularly busy")

In [45]:
result

['Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Regularly busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy',
 'Least busy

In [47]:
df_ords_prods_merge['busiest_day'] = result

In [49]:
df_ords_prods_merge['busiest_day'].value_counts(dropna = False)

busiest_day
Regularly busy    22417684
Busiest day        6204404
Least busy         3783953
Name: count, dtype: int64

## Question 2: Change busiest_day to plural busiest_days and add slowest days and average busy

In [63]:
result_2 = []
for value in df_ords_prods_merge["orders_day_of_week"]:
  if value == 0 or value == 1:  result_2.append("Busiest days")
  elif value == 4 or value == 3: result_2.append("Slowest days")
  else: result_2.append("Average busy days")

In [64]:
result_2

['Average busy days',
 'Average busy days',
 'Average busy days',
 'Average busy days',
 'Average busy days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Average busy days',
 'Average busy days',
 'Average busy days',
 'Average busy days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'S

## Question 3 check column value accuracy

In [69]:
df_ords_prods_merge['busiest_days'] = result_2

In [71]:
df_ords_prods_merge['busiest_days'].value_counts(dropna = False)

busiest_days
Average busy days    12916527
Busiest days         11864860
Slowest days          7624654
Name: count, dtype: int64

In [73]:
df_ords_prods_merge.head ()

,order_id,user_id,order_number,orders_day_of_week,orders_hour,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,match,price_range_loc,busiest_day,busiest_days
0,2539329,1,1,2,8,NaN,196,1,0,both,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Average busy days
1,2539329,1,1,2,8,NaN,14084,2,0,both,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,both,Mid-range product,Regularly busy,Average busy days
2,2539329,1,1,2,8,NaN,12427,3,0,both,Original Beef Jerky,23,19,4.4,both,Low-range product,Regularly busy,Average busy days
3,2539329,1,1,2,8,NaN,26088,4,0,both,Aged White Cheddar Popcorn,23,19,4.7,both,Low-range product,Regularly busy,Average busy days
4,2539329,1,1,2,8,NaN,26405,5,0,both,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,both,Low-range product,Regularly busy,Average busy days


## Question 4: “Most orders,” “Average orders,” and “Fewest orders.” Create a new column containing these labels called “busiest_period_of_day.”

In [78]:
df_ords_prods_merge['orders_hour'].value_counts()

orders_hour
10    2761862
11    2736242
14    2689251
15    2662246
13    2661057
12    2618613
16    2535313
9     2454273
17    2087721
8     1718182
18    1636566
19    1258352
20     976182
7      891082
21     795658
22     634247
23     402335
6      290500
0      218786
1      115706
5       87964
2       69377
4       53245
3       51281
Name: count, dtype: int64

In [80]:
result_3 = []
for value in df_ords_prods_merge["orders_hour"]:
  if value in [10, 11, 14, 15, 13, 12, 16, 9]: result_3.append("Most orders")
  elif value in [23, 6, 0, 1, 5, 2, 4, 3]: result_3.append("Fewest orders")
  else: result_3.append("Average orders")

In [82]:
result_3

['Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Avera

In [84]:
df_ords_prods_merge['busiest_period_of_day'] = result_3

## Question 5: Print the frequency for this new column

In [87]:
df_ords_prods_merge['busiest_period_of_day'].value_counts(dropna = False)

busiest_period_of_day
Most orders       21118857
Average orders     9997990
Fewest orders      1289194
Name: count, dtype: int64

In [89]:
df_ords_prods_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32406041 entries, 0 to 32406040
Data columns (total 19 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   user_id                 int64   
 2   order_number            int64   
 3   orders_day_of_week      int64   
 4   orders_hour             int64   
 5   days_since_prior_order  float64 
 6   product_id              int64   
 7   add_to_cart_order       int64   
 8   reordered               int64   
 9   _merge                  category
 10  product_name            object  
 11  aisle_id                int64   
 12  department_id           int64   
 13  prices                  float64 
 14  match                   category
 15  price_range_loc         object  
 16  busiest_day             object  
 17  busiest_days            object  
 18  busiest_period_of_day   object  
dtypes: category(2), float64(2), int64(10), object(5)
memory usage: 4.2+ GB


## Export as pickle file

In [92]:
df_ords_prods_merge.to_pickle(os.path.join(path, 'Data', 'Prepared Data', 'orders_products_merged_derived.pkl'))